# Data Analysis (Ex_Social)

## Import CSV into SQLite3

In [1]:
import csv, sqlite3
import pandas as pd

In [2]:
follows_df = pd.read_csv("../data/follows.csv")
follows_df.head(3)

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01


In [3]:
users_df = pd.read_csv("../data/users.csv")
users_df.head(3)

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor


In [18]:
conn = sqlite3.connect("ex_social.db")

In [13]:
follows_csv = "follows"
columns = ["user_id", "follows", "date"]
follows_db = f"Create table {follows_csv} ({', '.join(columns)})"

In [14]:
conn.execute(follows_db)

In [15]:
with open("../data/follows.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    data = [tuple(row) for row in reader]

In [16]:
insert_follows = f"Insert into {follows_csv} values ({', '.join(['?'] * len(columns))})"
for row in data:
    conn.execute(insert_follows, row)

In [17]:
conn.commit()

In [19]:
users_csv = "users"
columns = ["user_id", "first_name", "last_name", "house"]
users_db = f"Create table {users_csv} ({', '.join(columns)})"

conn.execute(users_db)

In [21]:
with open("../data/users.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    data = [tuple(row) for row in reader]

In [22]:
insert_users = f"Insert into {users_csv} values ({', '.join(['?'] * len(columns))})"
for row in data:
    conn.execute(insert_users, row)

In [23]:
conn.commit()
conn.close()

## Questions to Answer

In [4]:
dbfile = "../data/ex_social.db"
conn = sqlite3.connect(dbfile)

cur = conn.cursor()

### 1. How many users are there in each house?

In [5]:
def users_in_house(db):

    query = """
    SELECT u.house, COUNT(u.house)
    FROM users u
    GROUP BY (u.house)
    ORDER BY COUNT(u.house) DESC
    """

    db.execute(query)
    rows = db.fetchall()
    
    print(rows)

In [6]:
users_in_house(cur)

[('Gryffindor', 4), ('Slytherin', 2), ('Ravenclaw', 2), ('Hufflepuff', 1)]


### 2. List all following links that were created before September 1st, 1993

In [7]:
query = """
SELECT follows 
FROM follows
WHERE date < "1993-09-01"
"""

cur.execute(query)
rows = cur.fetchall()

rows

[('1',), ('1',), ('2',), ('1',), ('6',), ('1',)]

### 3. List all the following links established before September 1st 1993, but this time use the users first names

In [8]:
query = """
SELECT 
	f.follows,
	users.first_name 
FROM follows as f
JOIN users ON f.user_id = users.user_id 
WHERE date < "1993-09-01"
"""

cur.execute(query)
rows = cur.fetchall()

rows

[('1', 'Ron'),
 ('1', 'Hermonie'),
 ('2', 'Ginny'),
 ('1', 'Ginny'),
 ('6', 'Draco'),
 ('1', 'Luna')]

### 4. Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users fullname, number of followers

In [10]:
query = """
SELECT 
	(u.first_name || " " || u.last_name) as fullname,
	COUNT(f.follows)
FROM users u
JOIN follows f ON f.user_id = u.user_id
WHERE f.date < "1993-09-01"
GROUP BY fullname
ORDER BY COUNT(f.follows) DESC
"""

cur.execute(query)
rows = cur.fetchall()

rows

[('Ginny Weasley', 2),
 ('Ron Wesley', 1),
 ('Luna Lovegood', 1),
 ('Hermonie Granger', 1),
 ('Draco Malfoy', 1)]

### 5. List the number of users each user follows

In [11]:
query = """
SELECT 
	(u.first_name || " " || u.last_name) as fullname,
	COUNT(f.follows)
FROM users u
JOIN follows f ON f.user_id = u.user_id
GROUP BY fullname
ORDER BY u.user_id 
"""

cur.execute(query)
rows = cur.fetchall()

rows

[('Harry Potter', 5),
 ('Ron Wesley', 2),
 ('Hermonie Granger', 3),
 ('Ginny Weasley', 3),
 ('Draco Malfoy', 2),
 ('Luna Lovegood', 2),
 ('Cho Chang', 2),
 ('Cedric Diggory', 1)]

### 6. List all rows from follows where someone from one house follows someone from a different house. Please give the result with user names.

In [ ]:
query = """

"""

### 7. List all unrequited followings (i.e. where A follows B but B does not follow A)

In [13]:
query = """
SELECT
    f.user_id,
    f.follows
FROM follows f 
WHERE NOT EXISTS (
	SELECT *
	FROM follows f2 
	WHERE f.user_id = f2.follows AND f.follows = f2.user_id)
"""

cur.execute(query)
rows = cur.fetchall()

rows

[('3', '1'), ('4', '2'), ('5', '6'), ('7', '8'), ('3', '9'), ('4', '9')]